In [1]:
import pandas as pd 
import os 
import numpy as np

## Data Cleaning 

Found corrupt file entry in line number 8864 of Airports.csv for "Sim��_��__��_��___��_��__��_��_____n Bol��_��__��_��___��_��__��_��_____var International,Venezuela,10.599444,-66.9825" , manually corrected the file entry to "

In [2]:
# Data Files
data_dir = "/home/julian/git-repo/juliangdz/Multi-Scale-Network-Analysis-of-Flight-Data/data"
airport_dataframe = pd.read_csv(os.path.join(data_dir,'Airports.csv'))
flights_dataframe = pd.read_excel(os.path.join(data_dir,r'Flight Data.xlsx'))

In [3]:
airport_dataframe.head()

,id,label,country,Lat,Lon
0,BIN,Bamyan,Afghanistan,34.800000,67.816667
1,BST,Lashkar Gah,Afghanistan,31.582998,64.360000
2,CCN,Chaghcharan,Afghanistan,34.516701,65.250001
3,DAZ,Darwaz,Afghanistan,38.466667,70.883333
4,FAH,Farah,Afghanistan,32.391730,62.096819


In [4]:
flights_dataframe.head()

,Source,Source City,Source Country,Target,Target City,Target Country,Weight
0,FNC,Funchal,Portugal,PXO,Porto Santo,Portugal,9864
1,PXO,Porto Santo,Portugal,FNC,Funchal,Portugal,9864
2,AEP,Buenos Aires,Argentina,MVD,Montevideo,Uruguay,1463
3,MVD,Montevideo,Uruguay,AEP,Buenos Aires,Argentina,1463
4,AEP,Buenos Aires,Argentina,ROS,Rosario (AR),Argentina,2261


Assuming the information in the Flights Data excel is accurate 

### Additional Data Cleaning Required

- Found Discrepencies in Country column for Airports.csv . Will proceed to correct the discrepencies observed using the Information from Flights Data excel by matching the Source and Source Country with the ID of the Airports.csv to correct the Country 

In [5]:
def modify_country_information_based_on_infocus_countries(flights_df:pd.DataFrame,airports_df:pd.DataFrame,infocus_countries:dict):
    flights_df = flights_df.copy()
    airports_df = airports_df.copy()
    
    for key, value in infocus_countries.items():
        for country in value:
            flights_df.loc[flights_df['Source Country'] == country, 'Source Country'] = key
            flights_df.loc[flights_df['Target Country'] == country, 'Target Country'] = key
            airports_df.loc[airports_df['country'] == country, 'country'] = key

    return flights_df, airports_df

In [6]:
in_focus_countries = {
    "USA":["United States","USA","United States Minor Outlying Islands"],
    "China":["China","Hong Kong (sar) China","Macao (sar) China","Chinese Taipei"],
    "UK":["United Kingdom","UK"],
    "Australia":["Australia"]
}
modifed_flight_df,modified_airport_df = modify_country_information_based_on_infocus_countries(flights_dataframe,airport_dataframe,in_focus_countries)

In [7]:
# Save the Modified Flight and Airport Data 
modifed_flight_df.to_csv(os.path.join(data_dir,'modified_flight_data.csv'),index=False)
modified_airport_df.to_csv(os.path.join(data_dir,'modified_airport_data.csv'),index=False)

## Modify Flight Data to inlcude only Flights within the In-Focus Countries

- Should use only Internal Flights for each country so 'Source Country' == 'Target Country'

In [9]:
def get_domestic_flights_for_countries(flights_df:pd.DataFrame,in_focus_countries:dict):
    # Filter for only In-Focus Countries
    _countries_list = list(in_focus_countries.keys())
    domestic_flights_list = []
    for country in _countries_list:
        domestic_flights = flights_df[(flights_df['Source Country']==country)&(flights_df['Target Country']==country)]
        domestic_flights_df = domestic_flights_list.append(domestic_flights)
    domestic_flights_df = pd.concat(domestic_flights_list,ignore_index=True)
    return domestic_flights_df

In [10]:
domestic_flights_dataframe = get_domestic_flights_for_countries(modifed_flight_df,in_focus_countries)
domestic_flights_dataframe.head()

,Source,Source City,Source Country,Target,Target City,Target Country,Weight
0,SOV,Seldovia,USA,KEB,Nanwalek,USA,144
1,HOM,Homer,USA,SOV,Seldovia,USA,184
2,SOV,Seldovia,USA,HOM,Homer,USA,40
3,PGM,Port Graham,USA,HOM,Homer,USA,144
4,KEB,Nanwalek,USA,PGM,Port Graham,USA,144


In [12]:
domestic_flights_dataframe.to_csv(os.path.join(data_dir,'domestic_flights_data.csv'),index=False)

### Merge With Airport Data 

- Check if ID are matching or missing 
- Merge to include geo information for both source and target 
- Handle Missing or inconsistent data 
- Verify the integrity of the dataset ( no duplicate routes , reasonable flight weights )

In [15]:
def validate_and_merge_data_corrected(flight_data, airport_data):
    """
    Corrected function to validate and merge flight data with airport data to include geographical coordinates.
    Adjusted to handle the correct column names for latitude and longitude.

    Parameters:
    flight_data (DataFrame): The flight data DataFrame.
    airport_data (DataFrame): The airport data DataFrame.

    Returns:
    DataFrame: Merged flight data with geographical coordinates.
    """
    # Adjusting column names for latitude and longitude
    airport_data = airport_data.rename(columns={'Lat ': 'Lat', 'Lon': 'Lon'})

    # Checking for mismatches or missing data in airport IDs
    missing_source_ids = flight_data[~flight_data['Source'].isin(airport_data['id'])]
    missing_target_ids = flight_data[~flight_data['Target'].isin(airport_data['id'])]

    # Merging flight data with airport data to include coordinates
    merged_data = flight_data.merge(
        airport_data[['id', 'Lat', 'Lon']], left_on='Source', right_on='id', how='left'
    ).rename(columns={'Lat': 'Source Lat', 'Lon': 'Source Lon'}).drop('id', axis=1)

    merged_data = merged_data.merge(
        airport_data[['id', 'Lat', 'Lon']], left_on='Target', right_on='id', how='left'
    ).rename(columns={'Lat': 'Target Lat', 'Lon': 'Target Lon'}).drop('id', axis=1)

    return merged_data, missing_source_ids, missing_target_ids


def data_integrity_checks(merged_data):
    # Identifying rows with missing coordinates
    missing_coordinates = merged_data[merged_data[['Source Lat', 'Source Lon', 'Target Lat', 'Target Lon']].isna().any(axis=1)]

    # Counting the number of missing coordinate entries
    missing_count = missing_coordinates.shape[0]

    # Checking for duplicates in the flight data
    duplicates_count = merged_data.duplicated(subset=['Source', 'Target']).sum()

    # Checking for any anomalies in flight weights (e.g., negative values or extremely high values)
    weight_issues = merged_data[merged_data['Weight'] <= 0]

    # Counting the number of weight issues
    weight_issues_count = weight_issues.shape[0]

    return missing_count, duplicates_count, weight_issues_count, missing_coordinates.head()

def clean_merged_data(merged_data):
    """
    Cleans the merged flight data by handling missing or inconsistent data and verifying data integrity.

    Parameters:
    merged_data (DataFrame): The merged flight and airport data DataFrame.

    Returns:
    DataFrame: Cleaned merged flight data.
    """
    # Handling any missing or inconsistent data
    cleaned_data = merged_data.dropna(subset=['Source Lat', 'Source Lon', 'Target Lat', 'Target Lon'])

    # Removing duplicate routes
    cleaned_data = cleaned_data.drop_duplicates(subset=['Source', 'Target'])

    # Ensuring reasonable flight weights
    cleaned_data = cleaned_data[cleaned_data['Weight'] > 0]

    return cleaned_data

# Reapplying the corrected function
merged_flights_data_corrected, missing_source_ids_corrected, missing_target_ids_corrected = validate_and_merge_data_corrected(domestic_flights_dataframe, modified_airport_df)
data_integrity_checks(merged_flights_data_corrected)
# cleaned_flights_data_corrected = clean_merged_data(merged_flights_data_corrected)

# (missing_source_ids_corrected, missing_target_ids_corrected, cleaned_flights_data_corrected.head())


(51,
 9781,
 0,
      Source        Source City Source Country Target Target City  \
 82      HYL             Hollis            USA    KLW     Klawock   
 83      KLW            Klawock            USA    HYL      Hollis   
 84      HYL             Hollis            USA    KTN   Ketchikan   
 85      KTN          Ketchikan            USA    HYL      Hollis   
 1080    DFW  Dallas/Fort Worth            USA    ILE     Killeen   
 
      Target Country  Weight  Source Lat  Source Lon  Target Lat  Target Lon  
 82              USA     434         NaN         NaN   55.550000     -133.10  
 83              USA     434   55.550000 -133.100000         NaN         NaN  
 84              USA     434         NaN         NaN   55.333333     -131.65  
 85              USA     434   55.333333 -131.650000         NaN         NaN  
 1080            USA    6150   32.896944  -97.038056         NaN         NaN  )